> 回来准备在服务器上怕一下改程序，这里的内存不够大，或者尝试分布式读取

> 下面有两个batch_gennerator的函数，看一下是否只需要跑一个程序，还是需要用到两个？

In [2]:
import numpy as np
import pickle

In [10]:
# data generator function
def batch_generator(X, max_length=20, batch_size=1):
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    shuffle_index = np.arange(len(X))
    np.random.shuffle(shuffle_index)
    X = X[shuffle_index, :]
    
    while 1:
        index_batch = shuffle_index[batch_size*counter:batch_size*(counter+1)]
        Xtemp = [].extend(X[index_batch])
        status = np.sum([isinstance(e, list) for e in Xtemp])
        if (status==0):
            Xtemp = [Xtemp]
        X_batch = pad_sequences(Xtemp, maxlen=max_length)
        X_res = np.zeros((np.shape(X_batch)[0], maxlen, max_length), dtype=np.int)
        for k in np.range(np.shape(X_batch)[0]):
            X_res[k, 0, :] = X_batch[k, :]
            counter += 1
            yield(X_res)
            if (counter < number_of_batches):
                np.random.shuffle(shuffle_index)
                counter=0


In [ ]:
# read in Ubuntu forum conversation pickle file
with open("dataset.pkl", "rb") as f:
    data = pickle.load(f,encoding='utf-8')

In [3]:
# Load libraries
import keras
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Reshape, Dot, Concatenate, Multiply, Merge

context_length=np.max(list(map(len, data[0]['c'])))
#print(context_length)
response_length=np.max(list( map(len, data[0]['r'])))
#print(response_length)
# we can also set contenxt length to a shorter one
context_length=120

# dual encoder LSTM	
context_size = np.max(list(map(lambda x:  max(x) if len(x)>0 else 0, data[0]['c'])))
response_size = max(list(map(lambda x:  max(x) if len(x)>0 else 0, data[0]['r'])))
volcabulary_size=max(context_size, response_size)

embedding_dim=64
lstm_dim=64

response_length=120
response=Input(shape=((response_length,)), dtype='Int32', name='response_input')
response_embedded=Embedding(input_length=response_length, output_dim=embedding_dim, input_dim=volcabulary_size)(response)
response_lstm=LSTM(lstm_dim)(response_embedded)

context_length=120
context=Input(shape=((context_length,)), dtype='Int32', name='response_input')
context_embedded=Embedding(input_length=context_length, output_dim=embedding_dim, input_dim=volcabulary_size)(context)
context_lstm=LSTM(lstm_dim)(context)


x = Multiply()([context_lstm, response_lstm])
yhat = Dense(2, activation='softmax')(x)
model = Model(inputs=[context, response], outputs=yhat)

model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

NameError: name 'data' is not defined

In [ ]:

# data generators so that the model can run on regular PC
def data_gen(data, batch_size=100):
	from keras.preprocessing.sequence import pad_sequences
	contextRaw = data['c']
	responseRaw = data['r']
	yRaw = data['y']
    
	number_of_batches = len(contextRaw) # batch_size
	counter=0
    
	context_length=np.max(list(map(len, contextRaw)))#3    
	response_length=np.max(list( map(len, responseRaw)))#3
    
	context_length=120
	response_length=120

	while 1:   	 
    	lowerBound = batch_size*counter
    	upperBound = batch_size*(counter+1)
    	Ctemp = contextRaw[lowerBound : upperBound]
    	C_batch = pad_sequences(Ctemp, maxlen=context_length, padding='post')
    	C_res = np.zeros((batch_size, context_length), dtype=np.int)
   	 
    	Rtemp = responseRaw[lowerBound : upperBound]
    	R_batch = pad_sequences(Rtemp, maxlen=response_length, padding='post')
    	R_res = np.zeros((batch_size, response_length), dtype=np.int)
    	for k in np.arange(batch_size):
        	C_res[k, :] = C_batch[k, :]
        	R_res[k, :] = R_batch[k, :]
    	y_res= keras.utils.to_categorical(yRaw[lowerBound : upperBound])
    	counter += 1
    	yield([C_res.astype('float32'), R_res.astype('float32')], y_res.astype('float32'))
    	if (counter < number_of_batches):       	 
        		counter=0        	

# Now run the model
batch_size=100
model.fit_generator(data_gen(data[0], batch_size=batch_size), steps_per_epoch=len(data[0]['c']),epochs=1)#batch_size,


In [ ]:
# This is evaluation function
def recall(ypred, yactual, N=1):
    num_examples = float(len(y))
    num_correct = 0
    for predictions, actual in zip(ypred, yactual):
        if label in predictions[:N]:
            num_correct += 1
        return num_correct/num_examples